In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import seaborn as sns
from scipy import stats
import matplotlib as mpl
mpl.rc('figure', max_open_warning = 0)
os.chdir('/mnt/c/Users/engje/Documents/Data/')
import mplex_image         

In [ ]:
rootdir = '/mnt/c/Users/engje/Documents/Data/cmIF_2021-07-07_RS-mTMA'
os.chdir(rootdir)

# Table of contents <a name="contents"></a>

1. [Leiden](#l8)
[annotate](#lbar)

2. [subtype](#subt)  [color scheme](#color)

3. [Leiden celltypeing](#lbar)

4. [Expression results](#results)  [mibi](#resultsmibi)  [annotate mibi](#mibiann)

In [ ]:
df_cell = pd.read_csv('results_20211110_RS-mTMA-5_leiden0.5_gated_celltype.csv',index_col=0)

In [ ]:
s_out = '20211111_RS-mTMA-5'
df_man = pd.read_csv(f'{s_out}_GatedPositiveCellNames.csv',index_col=0)
#RS-mTMA-5_sceneA05 Sox9 >256
ls_index = df_cell.loc[(df_cell.index.str.contains('RS-mTMA-5_sceneA05')& (df_cell.SOX9 > 256))].index 
df_man.loc[ls_index,'celltype']='epithelial'
#H11
ls_index = df_cell.loc[(df_cell.index.str.contains('RS-mTMA-5_sceneH11')& (df_cell.SOX9 > 256))].index 
df_man.loc[ls_index,'celltype']='epithelial'
#ls_bad = ['stromal','epithelial']
#df_cell_scene =  df_cell.loc[(df_cell.index.str.contains('RS-mTMA-5_sceneH11'))]
#df_man_scene = df_man.loc[df_man.index.str.contains('RS-mTMA-5_sceneH11')].loc[df_cell_scene.index]
#print(df_man_scene.groupby('celltype').DAPI_X.count()/len(df_man_scene))
#ls_index = df_cell_scene.loc[(df_cell_scene.SOX9 > 256)].index 
#df_man.loc[ls_index,'celltype']='epithelial'
#ls_index_str = df_cell_scene.loc[(df_cell_scene.SOX9 < 256) & df_man_scene.celltype.isin(ls_bad)].index
#df_man.loc[ls_index_str,'celltype']='stromal'
#print(len(df_man.loc[(df_man.index.str.contains('RS-mTMA-5_sceneH11')&(df_man.celltype =='stromal'))]))

df_man['celltype3'] =  df_man.celltype.replace({'endothelial':'stromal','tumor':'epithelial'})
df_man['celltype'] = df_man.celltype.replace({'tumor':'epithelial'})

## subtype <a name="subt"></a>
 

[contents](#contents)

In [ ]:

#load annotation old
'''import re
df_a = pd.read_csv('RS-MYC-PTENfl_TMA_subgroups.csv')
df_a.loc[~df_a.GroupA.isna(),'Groups'] = 'stroma_rich'
df_a.loc[~df_a.GroupB.isna(),'Groups'] = 'stroma_poor'
df_a.loc[df_a.Groups.isna(),'Groups'] = 'not_tumor'
df_a['Xiaoyan'] = df_a.Xiaoyan.fillna(df_a.TMA)
df_a.index = df_a.TMA #drop(['oldE1','oldF6'],inplace=True)
d_a = dict(zip(df_a.index,df_a.Groups))
df_a['slide_scene'] = [f'RS-mTMA-5_scene{re.sub(r"[0-9]+", "", item)}{int(re.sub(r"[A-Z]+", "", item)):02d}' for item in df_a.index]
df_a['Group'] = df_a.Xiaoyan.map(d_a).fillna('not_tumor')      
'''

In [ ]:
s_sample = '20211110_RS-mTMA-5'
resolution = 0.5
s_date = s_sample.split('_')[0]
#load annotation
import re
df_a = pd.read_csv('MYC-PTENfl TMA subgroups1_better.csv')
df_a.loc[~df_a.GroupA.isna(),'Groups'] = 'stroma_rich'
df_a.loc[~df_a.GroupB.isna(),'Groups'] = 'stroma_poor'
df_a.loc[df_a.Groups.isna(),'Groups'] = 'not_tumor'
df_a['Xiaoyan'] = df_a.Xiaoyan.fillna(df_a.TMA)
df_a.index = df_a.TMA #drop(['oldE1','oldF6'],inplace=True)
d_a = dict(zip(df_a.index,df_a.Groups))
df_a['slide_scene'] = [f'RS-mTMA-5_scene{re.sub(r"[0-9]+", "", item)}{int(re.sub(r"[A-Z]+", "", item)):02d}' for item in df_a.index]
df_a['Group'] = df_a.Xiaoyan.map(d_a).fillna('not_tumor')    
#not tumor for neuNT
df_a.loc[df_a.Tumor_size_mm=='MYC-NeuNT','Group'] = 'not_tumor'

df_p = pd.read_csv(f'results_{s_sample}_leiden{resolution}_gated_celltype.csv',index_col=0)
df_p['slide_scene'] = [item.split('_cell')[0] for item in df_p.index]
df_p['slide'] = [item.split('_')[0] for item in df_p.index]
#annotate
d_a = dict(zip(df_a.slide_scene,df_a.Group))
df_p['Group'] = df_p.slide_scene.map(d_a)
df_p['celltype_l3'] =  df_p.gated_celltype.replace({'endothelial':'stromal'})
df_p['gated_celltype'] = df_p.gated_celltype.replace({'tumor':'epithelial'})

#filter out not tumor
df_man['Group'] = df_man.slide_scene.map(d_a)
df_man = df_man[df_man.Group!='not_tumor']
df_p = df_p[df_p.Group!='not_tumor']

In [ ]:
s_type = 'celltype3' # 'celltype' #'gated_celltype' # 'celltype_l3' #'leiden' #
ls_drop = ['RS-mTMA-5_sceneF03',
           #'RS-mTMA-5_sceneA11', 'RS-mTMA-5_sceneB06', 'RS-mTMA-5_sceneB10',
       #'RS-mTMA-5_sceneC06', 'RS-mTMA-5_sceneD04', 'RS-mTMA-5_sceneD09',
       #'RS-mTMA-5_sceneE02', 'RS-mTMA-5_sceneE06', 'RS-mTMA-5_sceneE12',
       #'RS-mTMA-5_sceneF04', 'RS-mTMA-5_sceneI10', 'RS-mTMA-5_sceneI12'
          ]
if s_type == 'celltype': 
    s_cell ='DAPI_X'
    df = df_man.loc[:,['DAPI_X',s_type,'slide_scene']].groupby([s_type,'slide_scene']).DAPI_X.count()/df_man.loc[:,['DAPI_X','slide_scene']].groupby(['slide_scene']).DAPI_X.count()
elif s_type == 'celltype3': 
    s_cell ='DAPI_X'
    df = df_man.loc[:,['DAPI_X',s_type,'slide_scene']].groupby([s_type,'slide_scene']).DAPI_X.count()/df_man.loc[:,['DAPI_X','slide_scene']].groupby(['slide_scene']).DAPI_X.count()
else:
    s_cell = 'CD31'
    df = df_p.loc[:,['CD31',s_type,'slide_scene']].groupby([s_type,'slide_scene']).CD31.count()/df_p.loc[:,['CD31','slide_scene']].groupby(['slide_scene']).CD31.count()
df_prop = df.unstack().T.fillna(0).drop(ls_drop)
ls_col = sorted(df_prop.columns.tolist())# ['endothelial','tumor','immune', 'stromal'] #
adata = sc.AnnData(df_prop.loc[:,ls_col])
adata.raw = adata
#reduce dimensionality
sc.tl.pca(adata, svd_solver='auto')
#fig,ax=plt.subplots(figsize=(3.5,5))
#sc.pl.highest_expr_genes(adata, n_top=48,ax=ax,save=f'Expression_{len(ls_col)}_{s_type}.png')
#plt.tight_layout()
#sc.pl.pca_variance_ratio(adata,n_pcs=32, log=True)

#umap
for n_neighbors in [6]: #,30
    results_file = f'{s_date}/{s_sample}_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.h5ad'
    
    if not os.path.exists(results_file):
        print('calc umap')
        # calculate neighbors 
        sc.pp.neighbors(adata, n_neighbors=n_neighbors) #, method='rapids'
        sc.tl.umap(adata)
    else:
        print('loading umap')
        adata = sc.read_h5ad(results_file)
    #color by markers
    
    figname = f"Umap_markers_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
    title=figname.split('.png')[0].replace('_',' ')
    sc.pl.umap(adata, color=ls_col,vmin='p1.5',vmax='p99.5',ncols=4,size=300,save=figname)
    #survival
    adata.obs['slide'] = adata.obs.index.tolist()
    adata.obs['Group'] = adata.obs.slide.map(d_a).fillna('unknown')
    #adata.obs['Group'] = adata.obs.index.map(d_a).fillna('unknown')
            
    figname = f"Umap_Group_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
    title=figname.split('.png')[0].replace('_',' ')
    fig,ax = plt.subplots(figsize=(3,2), dpi=200)
    sc.pl.umap(adata, color='Group',wspace=.25,save=figname,ax=ax,title=title,size=50)
    
    #save results
    #if not os.path.exists(results_file):
    adata.write(results_file)
    break

In [ ]:
#print(df_prop.loc['RS-mTMA-5_sceneH11'])
#df_prop.loc['RS-mTMA-5_sceneA05']

In [ ]:
#leiden
n_markers = len(ls_col)
for resolution in [0.2]: #0.3,
    results_file = f'{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}_leiden{resolution}.h5ad'
    sc.tl.leiden(adata,resolution=resolution)
    adata.write(results_file)
    fig,ax = plt.subplots(figsize=(2,1.5),dpi=200)
    figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}.png'
    sc.pl.umap(adata, color='leiden',ax=ax,title=figname.split('.png')[0].replace('_',' '),wspace=.25,save=figname,size=40)
    break

In [ ]:
#clustermap
%matplotlib inline
df_p = pd.DataFrame(data=adata.raw.X,index=adata.obs.index,columns=adata.var.index)
df_p['leiden'] = adata.obs['leiden']
d_replace = {}
df_plot = df_p.groupby('leiden').mean()
df_plot.index.name = f'leiden {resolution}'
g = sns.clustermap(df_plot,figsize=(3,3),cmap='viridis',z_score=1,vmin=-1.5,vmax=1.5
                   )
#g.fig.suptitle(f'leiden {resolution}',x=.9) 
g.savefig(f'{rootdir}/{s_date}/{s_sample}_clustermap_{s_type}_subtype_{resolution}.png',dpi=200)
marker_genes = df_p.groupby('leiden').mean().iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_p.groupby('leiden').mean().iloc[g.dendrogram_row.reordered_ind,:].index.tolist()

# Color scheme  <a name="color"></a>
 

[contents](#contents)

In [ ]:
d_color = {'SR_IR': (0.12156862745098039, 0.4666666666666667, 0.7058823529411765), #blue
 'SP': (1.0, 0.4980392156862745, 0.054901960784313725), #orange
 'SR_IP': (0.17254901960784313, 0.6274509803921569, 0.17254901960784313)} #green
d_replace = {'0':'SR_IR','1':'SR_IP','2':'SP','3':'SP'}

In [ ]:
#d_replace = {'3':'2'}
#d_replace = {'0':'SR_IR','1':'SR_IP','2':'SP','3':'SP'}
d_replace = {'0':'SR_IR','1':'SR_IP','2':'SP','3':'SP'}
df_plot = df_p.groupby(df_p.leiden.replace(d_replace)).mean()
df_plot.index.name = f'leiden {resolution}'
#fig,ax=plt.subplots(,dpi=300)
g = sns.clustermap(df_plot.loc[['SR_IR','SR_IP','SP'],:],cmap='viridis',figsize=(5,8), row_cluster=False, col_cluster=False, 
                   row_colors =df_plot.loc[['SR_IR','SR_IP','SP'],:].index.map(d_color),
                   cbar_pos = (0.1, 1.01, .1,.5),
                   cbar_kws = {'orientation':'horizontal'}
           )

plt.tight_layout()
g.savefig(f'{rootdir}/{s_date}/{s_sample}_heatmap_{s_type}_subtype_{resolution}_bars.png',dpi=300)

In [ ]:
fig,ax=plt.subplots(figsize=(3.2,2),dpi=300)
sns.heatmap(df_plot.loc[['SR_IR','SR_IP','SP'],:],cmap='viridis',ax=ax #,z_score=1,vmin=-1.5,vmax=1.5,figsize=(4,3.5),
                   )
plt.tight_layout()
fig.savefig(f'{rootdir}/{s_date}/{s_sample}_heatmap_{s_type}_subtype_{resolution}.png',dpi=300)

In [ ]:
df_p.mean()
#MIBIcelltype3
#epithelial    0.531436
#immune        0.232980
#stromal       0.235584

In [ ]:
from scipy import stats
df_p['Group'] = df_p.index.map(d_a)
df_tum = df_p[df_p.Group!='not_tumor']
confusion_matrix = pd.crosstab(df_tum.leiden.replace(d_replace),df_tum.Group)
fig,ax = plt.subplots(dpi=400,figsize=(3.5,3))
sns.heatmap(confusion_matrix.loc[['SR_IR','SR_IP','SP'],['stroma_rich','stroma_poor']], annot=True,ax=ax,cbar_kws={'label':'No. Cores'},cmap='viridis')
chi2, pvalue, dof, expected  = stats.chi2_contingency(confusion_matrix)
print(pvalue)
ax.set_title(f"Subtype vs. Histology (p = {pvalue:.2f})")
plt.tight_layout()
fig.savefig(f'./{s_date}/Chi_square_group_clustering_{s_type}.png')

In [ ]:
fig,ax = plt.subplots(dpi=200,figsize=(4.5,3))
sns.heatmap(expected,ax=ax,annot=True,cbar_kws={'label':'No. Cores'},cmap='viridis',vmin=3,vmax=31)
ax.set_xticklabels(['stroma_poor','stroma_rich'])
ax.set_xlabel('Group')
ax.set_ylabel('leiden')
ax.set_title('Expected')
plt.tight_layout()
fig.savefig(f'./{s_date}/Chi_square_group_clustering_expected_{s_type}.png')

In [ ]:
#sns.clustermap(df_prop) #df_p bad variable name! 
ls_drop = ['RS-mTMA-5_sceneF03']
#s_type = 'celltype' #'celltype3' #'leiden' #
#s_cell ='DAPI_X' # 'CD31'#  'none' #
if s_cell == 'CD31':
    df = df_p.loc[:,['CD31',s_type,'slide_scene']].groupby([s_type,'slide_scene']).CD31.count()/df_p.loc[:,['CD31','slide_scene']].groupby(['slide_scene']).CD31.count()
    df_prop = df.unstack().T.fillna(0).drop(ls_drop)
elif s_cell == 'DAPI_X':  
    df = df_man.loc[:,['DAPI_X',s_type,'slide_scene']].groupby([s_type,'slide_scene']).DAPI_X.count()/df_man.loc[:,['DAPI_X','slide_scene']].groupby(['slide_scene']).DAPI_X.count()
    df_prop = df.unstack().T.fillna(0).drop(ls_drop)
else:
    df_prop = df_p.loc[:,['immune','stromal','tumor']]#df.unstack().T.fillna(0).drop(ls_drop)

ls_drop_scene = df_prop[df_prop.index.isin(df_a[df_a.Group=='not_tumor'].slide_scene)].index
df_prop_plot = df_prop.drop(ls_drop_scene)#.drop('endothelial',axis=1)


d_color_new = dict(zip(['stroma_rich','stroma_poor'],['royalblue','orangered']))
row_colors = df_prop_plot.index.map(d_a).map(d_color_new)


g = sns.clustermap(df_prop_plot,figsize=(5,4),cmap='viridis',row_colors=row_colors,dendrogram_ratio=0.1,cbar_pos=(-.05, .87, .05, .15),#z_score=1
       method='ward')
#g.fig.suptitle(f'leiden {resolution}',x=.9) 
for label in d_color.keys():
    g.ax_row_dendrogram.bar(0, 0, color=d_color[label],
                            label=label, linewidth=0)
g.ax_row_dendrogram.legend(loc="right", ncol=1)
g.savefig(f'{rootdir}/{s_date}/{s_sample}_{s_type}_clustermap_Group_leiden{resolution}.png',dpi=300)
marker_genes = df_prop.iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_prop.iloc[g.dendrogram_row.reordered_ind,:].index.tolist()


In [ ]:
#zoom in 
#g = sns.clustermap(df_prop_plot,figsize=(5,24),cmap='viridis',row_colors=row_colors,dendrogram_ratio=0.1,cbar_pos=(-.05, .87, .05, .15),#z_score=1
#       method='ward')

In [ ]:
d_color

In [ ]:
df_prop_plot.index.map(adata.obs.leiden.astype('str').replace(d_replace))

In [ ]:
#d_color = dict(zip(sorted(adata.obs.leiden.replace(d_replace).unique()),sns.color_palette()[0:5]))
row_colors = df_prop_plot.index.map(adata.obs.leiden.astype('str').replace(d_replace)).map(d_color)

g = sns.clustermap(df_prop_plot,figsize=(5,4),cmap='viridis',row_colors=row_colors,dendrogram_ratio=0.1,
                   cbar_pos=(-.05, .87, .05, .15),       #z_score=0
                  method='ward')
#g.fig.suptitle(f'leiden {resolution}',x=.9) 
for label in adata.obs.leiden.replace(d_replace).unique():
    g.ax_row_dendrogram.bar(0, 0, color=d_color[label],
                            label=label, linewidth=0)
g.ax_row_dendrogram.legend(loc="right", ncol=1,framealpha=1)
g.savefig(f'{rootdir}/{s_date}/{s_sample}_{s_type}_clustermap_Group_leiden{resolution}_leiden.png',dpi=300)
marker_genes = df_prop.iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_prop.iloc[g.dendrogram_row.reordered_ind,:].index.tolist()

In [ ]:
d_cell3 = dict(zip(df_p.index,df_p.leiden))

## Annotate Leiden <a name="lbar"></a>

18 cell types


[contents](#contents)

In [ ]:
#n_neighbors = 16
#n_markers = 24
#resolution = 0.6
#adata = sc.read_h5ad(f'20211110_RS-mTMA-5_{n_neighbors}neighbors_{n_markers}markers_BGSubtracted_leiden{resolution}.h5ad')
#adata = sc.read_h5ad(f'20211110_RS-mTMA-5_s_15neighbors_24markers_BGSubtracted_leiden0.6.h5ad')

s_sample = '20211110_RS-mTMA-5' #'20211110_RS-mTMA-5_s' # 
s_type = 'BGSubtracted'
s_date = s_sample.split('_')[0]
n_neighbors = 16 #15 #16
n_markers = 24 #23 #24
resolution = 0.6
results_file = f'{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}_leiden{resolution}.h5ad'
adata = sc.read_h5ad(results_file)

In [ ]:
results_file

In [ ]:
adata.var.index

In [ ]:
#figname=f'Umap_{n_neighbors}_neighbors_{n_markers}_markers.png'
#title=figname.split('.png')[0].replace('_',' ')
#fig = sc.pl.umap(adata, color=adata.var.index,vmin='p1.5',vmax='p99.5',ncols=4,save=figname)


In [ ]:
fig,ax = plt.subplots(figsize=(3,3),dpi=300) #bad clust '19','20','17'
figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}.png'
sc.pl.umap(adata, color='leiden',ax=ax,title=figname.split('.png')[0].replace('_',' '),wspace=.25,save=figname,)

In [ ]:
# weird ones
'''
s_slide = 'RS-mTMA-5_sceneI08' #
#s_slide = 'RS-mTMA-5_sceneI02'
#for s_slide in adata.obs.slide.unique():
adata.obs['test'] = (adata.obs.slide == s_slide).replace({True:1,False:0})
sc.pl.umap(adata, color='test',title=s_slide)
'''

In [ ]:
# weird ones
s_slide = '14'#'10' #
#s_slide = 'RS-mTMA-5_sceneI02'
#for s_slide in adata.obs.slide.unique():
adata.obs['test'] = (adata.obs.leiden == s_slide).replace({True:1,False:0})
sc.pl.umap(adata, color='test',title=s_slide)

In [ ]:
if results_file == '20211110_RS-mTMA-5_s_15neighbors_24markers_BGSubtracted_leiden0.6.h5ad':
    d_celltypes = {}
elif results_file == '20211110_RS-mTMA-5_s_15neighbors_23markers_BGSubtracted_leiden0.6.h5ad':
    print('good')
    d_celltypes = {}
else:
    d_celltypes = {'18':'dendritic_imm.','11':'artifact','17':'artifact','19':'artifact','20':'artifact',
               '7':'Myoepithelial',
               '10':'Ecad-_tum','14':'Ecad-_tum',
               '0':'Mix_imm_endothel.','3':'Mix_fb_endothel.','6':'NOS_stroma',
               '16':'NOS_stroma','12':'Poorly_diff_tum',
                   '13':'Ecad_low_tum','5':'CK5_SOX9_tum',
      '4':'CK5/SOX9_low_tum','9':'CK5/SOX9_low_tum',   '8':'Ki67_pMYC_hi_tum', 
                   '15':'EpCAM_hi_tum',  '1':'CK5_Ki67_tum', '2':'Ecad_Ki67_hi_tum'} #'9':'EpCAM_hi_tum'
#pd.DataFrame(pd.Series(d_celltypes)).to_csv('annotation_leiden_celltype.csv')
#pd.DataFrame(pd.Series(d_celltypes)).to_csv('annotation_leiden_scale_celltype.csv')

In [ ]:
df_p = pd.DataFrame(data=adata.raw.X,index=adata.obs.index,columns=adata.var.index)
df_p['leiden'] = adata.obs['leiden']
if results_file == '20211110_RS-mTMA-5_s_15neighbors_24markers_BGSubtracted_leiden0.6.h5ad':
    ls_drop_index = ['22','21','19','18']
    tum_clust = []
    imm_clust = []
    endo_clust = []
    str_clust = []
    ls_merge = ['20','6']
    df_p.loc[df_p.leiden.isin(ls_merge),'leiden'] = '6'
elif results_file=='20211110_RS-mTMA-5_s_15neighbors_23markers_BGSubtracted_leiden0.6.h5ad':
    ls_drop_index = ['19','18']
    tum_clust = []
    imm_clust = []
    endo_clust = []
    str_clust = []

else:
    ls_drop_index = ['19','20','17','11','9','14','16',]
    tum_clust = ['10','12','7','14','5','4','9','8','15','1','2','13']
    imm_clust = []
    endo_clust = []
    str_clust = ['0','3','6','18']
    df_p = df_p.drop('CD8',axis=1)
    ls_merge = ['10','14']
    df_p.loc[df_p.leiden.isin(ls_merge),'leiden'] = '10'
    ls_merge = ['4','9']
    df_p.loc[df_p.leiden.isin(ls_merge),'leiden'] = '4'
    ls_merge = ['16','6']
    df_p.loc[df_p.leiden.isin(ls_merge),'leiden'] = '6'

In [ ]:
#clustermap
b_annot = False
#df_p = pd.DataFrame(data=adata.raw.X,index=adata.obs.index,columns=adata.var.index)
#df_p['leiden'] = adata.obs['leiden']
d_replace = {}
d_replace.update(dict(zip(tum_clust,[f'{item}\nep' for item in tum_clust])))
d_replace.update(dict(zip(imm_clust,[f'{item}\nim' for item in imm_clust])))
d_replace.update(dict(zip(str_clust,[f'{item}\ns' for item in str_clust])))
d_replace.update(dict(zip(endo_clust,[f'{item}\nen' for item in endo_clust])))
if b_annot:
    df_plot = df_p.groupby('leiden').mean().drop(ls_drop_index).dropna().rename(d_celltypes)#(d_replace)#d_replace
    #df_plot = df_p.groupby('leiden').mean().drop(ls_drop_index).dropna().rename(d_replace)#d_replace
else:
    df_plot = df_p.groupby('leiden').mean().drop(ls_drop_index).dropna()
df_plot.index.name = f'leiden {resolution}'
g = sns.clustermap(df_plot,z_score=1,figsize=(10,8),cmap='viridis',method='ward',cbar_pos=(.01, .91, .05, .15),
                   vmin=-2,vmax=2,dendrogram_ratio=0.1)
g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize = 16)
g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_ymajorticklabels(), fontsize = 16)

g.savefig(f'{rootdir}/figures/{s_sample}_clustermap_leiden_{resolution}.png',dpi=300)
marker_genes = df_plot.iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_plot.iloc[g.dendrogram_row.reordered_ind,:].index.tolist()

In [ ]:
#df_plot.to_csv(f'cycIF_celltype_vs_expression_for_Nick.csv')

In [ ]:
fig,ax=plt.subplots(figsize=(2,4),dpi=200)
df_p[~df_p.leiden.isin(ls_drop_index)].groupby('leiden').count().loc[categories_order[::-1]].iloc[:,1].plot(kind='barh',title='Cell Count',ax=ax)
plt.tight_layout()
fig.savefig(f'./{s_date}/barplot_{n_neighbors}neighbors_{len(marker_genes)}markers_leiden{resolution}.png')

In [ ]:
df_p['Group'] = [d_a[item.split('_cell')[0]] for item in df_p.index]
df_p['slide_scene'] = [item.split('_cell')[0] for item in df_p.index]
df_p['leiden_subtype'] = df_p.slide_scene.map(d_cell3).replace( {'0':'SR_IR','1':'SR_IP','2':'SP','3':'SP'}) #d_replace = {'0':'SR_IR','1':'SR_IP','2':'SP','3':'SP'}
df_p['countme'] = True
s_clust = '2'
#df_p['core'] = df_p.merge(df_norm.loc[:,['core']],left_index=True,right_index=True).core
df_p.groupby(['leiden','slide_scene']).count().loc[s_clust].iloc[:,1].sort_values(ascending=False)[0:10]

In [ ]:
#plot one leiden 

b_start = False #True
if b_start:
    from mplex_image import analyze
    colors = mpl.cm.tab20b.colors + mpl.cm.tab20c.colors
    df_m = df_p.merge(df_man.loc[:,['DAPI_X','DAPI_Y','celltype']],left_index=True,right_index=True)
    df_pos = analyze.celltype_to_bool(df_m,'leiden')
    df_pos.columns = [str(item) for item in df_pos.columns]
    df_data = df_m

s_slide = 'RS-mTMA-5_sceneB08' #'RS-mTMA-5_sceneC08' #'RS-mTMA-5_sceneG02' #'RS-mTMA-5_sceneG06' #'RS-mTMA-5_sceneG12' #'RS-mTMA-5_sceneI06' #'RS-mTMA-5_sceneA05' #'RS-mTMA-5_sceneC03' #'RS-mTMA-5_sceneI01' #'RS-mTMA-5_sceneH07' #'RS-mTMA-5_sceneC10' #'RS-mTMA-5_sceneA01' #

#plot negative cells
df_scene = df_data[df_data.index.str.contains(s_slide)]
if len(df_scene) ==0:
    print('not included')
else:
    fig,ax = plt.subplots(figsize=(5,4.5),dpi=200)
    ax.scatter(data=df_scene,x='DAPI_X',y='DAPI_Y',color='silver',s=0.1,label=f'')
    #for idxs, s_color_int in enumerate(range(len(df_pos.columns))):
    for idxs, s_color in enumerate(df_pos.columns):
        s_color = s_clust
        s_color = str(s_color)
        if len(df_data[(df_data.slide_scene==s_slide) & (df_pos.loc[:,s_color])])>=1:
            #plot positive cells
            ls_index = df_data[(df_data.slide_scene==s_slide) & (df_pos.loc[:,s_color])].index
            ax.scatter(data=df_data.loc[ls_index],x='DAPI_X',y='DAPI_Y',label=f'{s_color}',s=0.1,color=colors[idxs])
        break
    ax.set_title(f"{s_slide}", fontsize=16) # \n {d_a[s_slide]}
    ax.axis('equal')
    ax.set_ylim(ax.get_ylim()[::-1])
    #ax.set_xticklabels('')
    #ax.set_yticklabels('')
    #break
    plt.legend(markerscale=10,framealpha=.5) 
    #fig.savefig(f'{codedir}/{s_date}/{s_slide}_leiden{resolution}_scatterplot.png')


In [ ]:
d_color

In [ ]:
%matplotlib inline
fig,ax=plt.subplots(figsize=(2.5,3.8),dpi=200)
df_wide = df_p.dropna().groupby(['leiden','leiden_subtype']).count().countme.unstack().loc[categories_order[::-1]]
df_wide_s = df_wide/df_p.groupby('leiden_subtype').count().countme
df_wide_s.plot(kind='barh',ax=ax,width=.8,title='Fraction Cells',color=d_color)
ax.set_title('Fraction Cells',fontsize=9)
ax.legend(bbox_to_anchor=(1,.9))
plt.tight_layout()
fig.savefig(f'./{s_date}/barplot3_{n_neighbors}neighbors_{len(marker_genes)}markers_leiden{resolution}.png')

In [ ]:
df_p.loc[df_p.leiden.isin(tum_clust),'leiden_celltype'] = 'epithelial'
df_p.loc[df_p.leiden.isin(str_clust),'leiden_celltype'] = 'stromal'
df_p.loc[df_p.leiden.isin(imm_clust),'leiden_celltype'] = 'immune'
df_p.loc[df_p.leiden.isin(endo_clust),'leiden_celltype'] = 'endothelial'
#df_p.loc[ls_drop_index,'leiden_celltype'] = 'artifact'
#df_p.leiden_celltype = df_p.leiden_celltype.fillna('NOS')
adata.obs['leiden_celltype'] = df_p.leiden_celltype
fig,ax = plt.subplots(figsize=(3,3),dpi=200)
figname=f'_leiden_celltype.png'
sc.pl.umap(adata, color='leiden_celltype',ax=ax,title='Annotated Leiden celltypes',save=figname,wspace=0.25) #

In [ ]:
adata.obs['celltype'] = df_man.merge(adata.to_df(),left_index=True,right_index=True,how='right').celltype.replace({'immune':'stromal','endothelial':'stromal'})
fig,ax = plt.subplots(figsize=(3,3),dpi=200)
figname=f'manual__celltype.png'
sc.pl.umap(adata, color='celltype',ax=ax,title='Manual gating',save=figname,wspace=0.25) #

In [ ]:
#the cell types
df_bar = df_p.dropna().groupby(['leiden','leiden_subtype']).count().countme.unstack()
g = sns.clustermap(df_bar.drop(ls_drop_index).rename(d_celltypes),z_score=0,cmap='viridis',dendrogram_ratio=0.15,figsize=(8,8))
g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize = 20)
g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_ymajorticklabels(), fontsize = 16)
g.savefig(f'./figures/Leiden_celltypes_per_leiden_subtype.png',dpi=300)

In [ ]:
#the cell types
df_bar = df_p.dropna().groupby(['leiden','leiden_subtype']).count().countme.unstack()
g = sns.clustermap(df_bar.drop(ls_drop_index).rename(d_celltypes),z_score=1,
                   cmap='viridis',dendrogram_ratio=0.15,figsize=(8,8))
g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize = 20)
g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_ymajorticklabels(), fontsize = 16)
g.savefig(f'./figures/Leiden_celltypes_per_leiden_subtype_noz.png',dpi=300)

In [ ]:
#the cores
d_color = dict(zip(['stroma_rich','stroma_poor'],['gold','mediumblue']))
df_prop_plot = (df_p.dropna().groupby(['slide_scene','leiden']).count().countme.unstack().T/(df_p.dropna().groupby(['slide_scene']).count().countme)).T
df_prop_plot = df_prop_plot.drop(ls_drop_index,axis=1).rename(d_celltypes,axis=1).T
row_colors = df_prop_plot.columns.map(d_a).map(d_color)
g = sns.clustermap(df_prop_plot,col_colors=row_colors,cbar_pos=(.01, .91, .05, .15),
                   vmin=-1.5,vmax=1.5,z_score=1,
                   method='ward',
                  cmap='viridis',dendrogram_ratio=0.1,figsize=(6.5,5.5))
g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_ymajorticklabels(), fontsize = 12)
g.savefig(f'{rootdir}/figures/{s_sample}_clustermap_leiden_by_patient_histology_{resolution}.png',dpi=300)

In [ ]:
d_color = {'SR_IR': (0.12156862745098039, 0.4666666666666667, 0.7058823529411765), #blue
 'SP': (1.0, 0.4980392156862745, 0.054901960784313725), #orange
 'SR_IP': (0.17254901960784313, 0.6274509803921569, 0.17254901960784313)} #green
d_replace = {'0':'SR_IR','1':'SR_IP','2':'SP','3':'SP'}

In [ ]:
d_color = {'0': (0.12156862745098039, 0.4666666666666667, 0.7058823529411765), #blue SP
 '2': (1.0, 0.4980392156862745, 0.054901960784313725), #orange
 '3': (1.0, 0.4980392156862745, 0.054901960784313725), #orange
 '1': (0.17254901960784313, 0.6274509803921569, 0.17254901960784313)} #green
#d_replace = {'0':'SR_IR','1':'SR_IP','2':'SP','3':'SP'}
df_prop_plot = (df_p.dropna().groupby(['slide_scene','leiden']).count().countme.unstack().T/(df_p.dropna().groupby(['slide_scene']).count().countme)).T
row_colors = df_prop_plot.index.map(d_cell3).map(d_color)
df_prop_plot = df_prop_plot.drop(ls_drop_index,axis=1).rename(d_celltypes,axis=1).T
g = sns.clustermap(df_prop_plot,col_colors=row_colors,cbar_pos=(.01, .91, .05, .15),
                   vmin=-1.5,vmax=1.5,z_score=1,
                   method='ward',
                   cmap='viridis',dendrogram_ratio=0.1,figsize=(6.5,5.5))
g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_ymajorticklabels(), fontsize = 12)
g.savefig(f'{rootdir}/{s_date}/{s_sample}_clustermap_leiden_by_patient_Subtype_{resolution}.png',dpi=300)

In [ ]:
df_out = df_p.merge(df_man.loc[:,['celltype3']],left_index=True,right_index=True) #,'DAPI_X','DAPI_Y'

In [ ]:
#old
d_sub = {'RS-mTMA-5_sceneA01': '0', 'RS-mTMA-5_sceneA04': '0', 'RS-mTMA-5_sceneA05': '0', 'RS-mTMA-5_sceneA06': '0', 'RS-mTMA-5_sceneA07': '1',
 'RS-mTMA-5_sceneA08': '0', 'RS-mTMA-5_sceneA09': '1', 'RS-mTMA-5_sceneB01': '0', 'RS-mTMA-5_sceneB04': '1', 'RS-mTMA-5_sceneB05': '1',
 'RS-mTMA-5_sceneB07': '1', 'RS-mTMA-5_sceneB08': '2', 'RS-mTMA-5_sceneB09': '1', 'RS-mTMA-5_sceneB11': '0', 'RS-mTMA-5_sceneC01': '0',
 'RS-mTMA-5_sceneC03': '0', 'RS-mTMA-5_sceneC04': '1', 'RS-mTMA-5_sceneC05': '0', 'RS-mTMA-5_sceneC07': '1', 'RS-mTMA-5_sceneC08': '0',
 'RS-mTMA-5_sceneC09': '0', 'RS-mTMA-5_sceneC10': '0', 'RS-mTMA-5_sceneC11': '2', 'RS-mTMA-5_sceneC12': '1', 'RS-mTMA-5_sceneD01': '1',
 'RS-mTMA-5_sceneD03': '0', 'RS-mTMA-5_sceneD05': '1', 'RS-mTMA-5_sceneD06': '1', 'RS-mTMA-5_sceneD07': '2', 'RS-mTMA-5_sceneD08': '0',
 'RS-mTMA-5_sceneD10': '1', 'RS-mTMA-5_sceneD11': '2', 'RS-mTMA-5_sceneE03': '0', 'RS-mTMA-5_sceneE04': '0', 'RS-mTMA-5_sceneE05': '1',
 'RS-mTMA-5_sceneE07': '2', 'RS-mTMA-5_sceneE08': '0', 'RS-mTMA-5_sceneE09': '1', 'RS-mTMA-5_sceneE10': '1', 'RS-mTMA-5_sceneE11': '1',
 'RS-mTMA-5_sceneF02': '0', 'RS-mTMA-5_sceneF05': '1', 'RS-mTMA-5_sceneF07': '0', 'RS-mTMA-5_sceneF08': '2', 'RS-mTMA-5_sceneF09': '2',
 'RS-mTMA-5_sceneF10': '0', 'RS-mTMA-5_sceneF11': '2', 'RS-mTMA-5_sceneF12': '0', 'RS-mTMA-5_sceneG01': '1', 'RS-mTMA-5_sceneG02': '0',
 'RS-mTMA-5_sceneG04': '0', 'RS-mTMA-5_sceneG05': '0', 'RS-mTMA-5_sceneG06': '2', 'RS-mTMA-5_sceneG08': '2', 'RS-mTMA-5_sceneG09': '2',
 'RS-mTMA-5_sceneG10': '2', 'RS-mTMA-5_sceneG11': '2', 'RS-mTMA-5_sceneG12': '2', 'RS-mTMA-5_sceneH03': '0', 'RS-mTMA-5_sceneH04': '2',
 'RS-mTMA-5_sceneH05': '0', 'RS-mTMA-5_sceneH06': '2', 'RS-mTMA-5_sceneH07': '0', 'RS-mTMA-5_sceneH08': '0', 'RS-mTMA-5_sceneH09': '1',
 'RS-mTMA-5_sceneH10': '0', 'RS-mTMA-5_sceneH11': '0', 'RS-mTMA-5_sceneH12': '2', 'RS-mTMA-5_sceneI01': '1', 'RS-mTMA-5_sceneI02': '1',
 'RS-mTMA-5_sceneI03': '0', 'RS-mTMA-5_sceneI04': '0', 'RS-mTMA-5_sceneI05': '0', 'RS-mTMA-5_sceneI06': '0', 'RS-mTMA-5_sceneI07': '2',
 'RS-mTMA-5_sceneI08': '1', 'RS-mTMA-5_sceneI09': '0', 'RS-mTMA-5_sceneI11': '2'}

In [ ]:
s_out = 'results_20211129_RS-mTMA-5_Celltype_Subtype_Group.csv'
s_out = 'results_20220107_RS-mTMA-5_Celltype_Subtype_Group.csv'
s_out = 'results_20220119_RS-mTMA-5_Celltype_Subtype_Group.csv'
s_out = 'results_20220121_RS-mTMA-5_Celltype_Subtype_Group.csv'
if not os.path.exists(s_out):
    print('saving csv')
    df_out.to_csv(s_out)

## visualize

In [ ]:
from mplex_image import analyze
#df_p = pd.read_csv(f'results_{s_sample}_leiden{resolution}_gated_celltype.csv',index_col=0)
df_p['slide_scene'] = [item.split('_cell')[0] for item in df_p.index]
#df_p['slide'] = [item.split('_')[0] for item in df_p.index]
#annotate
#d_a = dict(zip(df_a.slide_scene,df_a.Group))
#df_p['Group'] = df_p.slide_scene.map(d_a)
df_m = df_p.merge(df_man.loc[:,['DAPI_X','DAPI_Y','celltype']],left_index=True,right_index=True)
df_pos = analyze.celltype_to_bool(df_m,'leiden')
df_pos.columns = [str(item) for item in df_pos.columns]

In [ ]:
sns.palplot(mpl.cm.Paired.colors)
d_color = {'Ecad-_tum':mpl.cm.Paired.colors[0],
           'CK5/SOX9_low_tum':mpl.cm.Paired.colors[8],
           'Mix_fb_endothel.':mpl.cm.Paired.colors[10],
           'CK5_SOX9_tum':mpl.cm.Paired.colors[9],
           'Ecad_Ki67_hi_tum':mpl.cm.Paired.colors[3],
           'Myoepithelial':mpl.cm.Paired.colors[1],
           'Ki67_pMYC_hi_tum':mpl.cm.Paired.colors[5],
           'CK5_Ki67_tum':mpl.cm.Paired.colors[7],
           'Mix_imm_endothel.':mpl.cm.Paired.colors[4],}
d_color.update({'artifact':'gray'})

#set(d_celltypes.keys()) - set(ls_rare)
ls_rare = ['20', '15', '14', '9', '19', '16', '6', '13', '12', '11','18']
ls_com = ['17', '4', '10', '1', '8', '0', '7', '5', '3', '2']#'17',
#ls_com = d_celltypes.keys()
#set(['10', '12', '13', '15', '16', '17', '19', '20', '6', '9']).intersection(set(['10', '12', '13', '15', '16', '19', '20', '6']))
d_com = dict(zip(ls_com,[d_celltypes[item] for item in ls_com]))
#d_color = dict(zip(set([item for key, item in d_com.items()]),mpl.cm.tab10.colors[::-1]))
#d_color = dict(zip(set([item for key, item in d_celltypes.items()]),mpl.cm.tab20.colors[::-1]))
d_color.update({'artifact':'gray'})

In [ ]:
#plot all groups spatially - leiden 
ls_out = []

#for s_slide in sorted(set(df_p.slide_scene)):
for s_slide in ['RS-mTMA-5_sceneB08','RS-mTMA-5_sceneI03','RS-mTMA-5_sceneH11','RS-mTMA-5_sceneA05']:
    s_slide = 'RS-mTMA-5_sceneB07'
    #s_slide = 'RS-mTMA-5_sceneA09'
    fig,ax = plt.subplots(figsize=(8,4.5),dpi=300)
    #plot negative cells
    df_scene = df_m[df_m.index.str.contains(s_slide)]
    ax.scatter(data=df_scene,x='DAPI_X',y='DAPI_Y',color='silver',s=0.1,label=f'')
    #for idxs, s_color_int in enumerate(range(len(df_pos.columns))):
    for idxs, s_color in enumerate(d_com.keys()):
        s_color = str(s_color)
        if len(df_m[(df_m.slide_scene==s_slide) & (df_pos.loc[:,s_color])])>=10:
            #plot positive cells
            ax.scatter(data=df_m[(df_m.slide_scene==s_slide) & (df_pos.loc[:,s_color])],x='DAPI_X',y='DAPI_Y',
                       label=d_celltypes[s_color].replace('artifact','_artifact'),
                       s=0.1,color=d_color[d_celltypes[s_color]])
        else:
            ls_out.append(s_color)
    ax.set_title(f"{s_slide} \n {d_a[s_slide]}", fontsize=16)
    ax.axis('equal')
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)
    plt.tight_layout()
    plt.legend(markerscale=15,framealpha=.5,bbox_to_anchor=(.685,.8),fontsize=8) 
    fig.savefig(f'{rootdir}/SpatialPlots/{s_slide}_leiden{resolution}_scatterplot.png')
    break

In [ ]:
df_pos = analyze.celltype_to_bool(df_m,'celltype')
df_pos.columns

In [ ]:
#plot all groups spatially - leiden 
ls_out = []
df_pos = analyze.celltype_to_bool(df_m,'celltype')
d_color = {'endothelial':'C3', 'epithelial':'C0', 'immune':'C2', 'stromal':'C1'}
d_color = {'endothelial':'C1', 'epithelial':'C0', 'immune':'C2', 'stromal':'C1'}
d_color = {'endothelial':'C3', 'epithelial':'C1', 'immune':'C0', 'stromal':'C2'}
for s_slide in sorted(set(df_p.slide_scene)):
    s_slide = 'RS-mTMA-5_sceneB07'
    s_slide = 'RS-mTMA-5_sceneA09'
    s_slide = 'RS-mTMA-5_sceneC01' #SR
    s_slide = 'RS-mTMA-5_sceneD07'#SP
    fig,ax = plt.subplots(figsize=(8,4.5),dpi=300)
    #plot negative cells
    df_scene = df_m[df_m.index.str.contains(s_slide)]
    ax.scatter(data=df_scene,x='DAPI_X',y='DAPI_Y',color='silver',s=0.1,label=f'')
    #for idxs, s_color_int in enumerate(range(len(df_pos.columns))):
    for idxs, s_color in enumerate(df_pos.columns):
        if len(df_m[(df_m.slide_scene==s_slide) & (df_pos.loc[:,s_color])])>=10:
            #plot positive cells
            ax.scatter(data=df_m[(df_m.slide_scene==s_slide) & (df_pos.loc[:,s_color])],x='DAPI_X',y='DAPI_Y',
                       label=s_color.replace('endothelial','endothelial'),
                       s=0.1,color=d_color[s_color])
        else:
            ls_out.append(s_color)
    ax.set_title(f"{s_slide} \n {d_a[s_slide]}", fontsize=16)
    ax.axis('equal')
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)
    plt.tight_layout()
    plt.legend(markerscale=15,framealpha=.5,bbox_to_anchor=(.75,.7)) 
    fig.savefig(f'{rootdir}/{s_date}/{s_slide}_gated_celltype3_scatterplot.png')
    break

In [ ]:
df_prop.sort_values(by='immune',ascending='False')
ls_index = df_prop.sort_values(by='stromal',ascending='False')[0:20].index

In [ ]:
s_slide = 'RS-mTMA-5_sceneB08'
s_slide = 'RS-mTMA-5_sceneC08'
s_slide = 'RS-mTMA-5_sceneF02'
s_slide = 'RS-mTMA-5_sceneC01'
s_slide = 'RS-mTMA-5_sceneB08'
s_slide = 'RS-mTMA-5_sceneC11'
df_a[df_a.slide_scene == s_slide]
df_a[(df_a.slide_scene.isin(ls_index)) & (df_a.Group=='stroma_poor')]

## load  results mouse TMA <a name="results"></a>

celltype, subtype and group

- subtype versus marker expression

- histology group versus marker expression


[contents](#contents)

In [ ]:
d_color = {'SR_IR': (0.12156862745098039, 0.4666666666666667, 0.7058823529411765), #blue
 'SP': (1.0, 0.4980392156862745, 0.054901960784313725), #orange
 'SR_IP': (0.17254901960784313, 0.6274509803921569, 0.17254901960784313)} #green
d_replace = {'0':'SR_IR','1':'SR_IP','2':'SP','3':'SP'}

In [ ]:
#df_res = pd.read_csv('results_20211129_RS-mTMA-5_Celltype_Subtype_Group.csv',index_col=0)
#df_res = pd.read_csv('results_20220107_RS-mTMA-5_Celltype_Subtype_Group.csv',index_col=0)
#df_res = pd.read_csv('results_20220119_RS-mTMA-5_Celltype_Subtype_Group.csv',index_col=0)
df_res = pd.read_csv('results_20220121_RS-mTMA-5_Celltype_Subtype_Group.csv',index_col=0)
df_res.dropna(inplace=True) #drop not tumor samples
#df_res['leiden_subtype'] =  df_res.leiden_subtype.astype('int') # df_res.leiden_man3.astype('int') # 
#df_res['DAPI_X'] = df_res.DAPI_X.astype('int')
#df_res['DAPI_Y'] = df_res.DAPI_Y.astype('int')
ls_marker = df_res.columns[df_res.dtypes=='float64'].tolist()
#old 1129df_res['leiden_subtype_group'] = df_res.leiden_subtype.replace({0: 'stroma',1: 'immune',3: 'epithelial',2: 'epithelial'})
#olddf_res['leiden_subtype_group'] =  df_res.leiden_subtype.replace({0:'stroma_rich',1:'stroma_rich',2:'stroma_poor'})
#d_replace = {0:'imm-str_rich',1:'str_mid',2:'str_poor',3:'str_poor'}
#d_replace = {0:'SR_IR',1:'SR_IP',2:'SP',3:'SP'}

df_res['leiden_subtype_group'] =  df_res.leiden_subtype#.replace(d_replace)

In [ ]:
#df_res['celltype2'] = df_res.celltype.replace({'immune':'stromal','endothelial':'stromal'})
df_res['celltype2'] = df_res.celltype3.replace({'immune':'stromal'})
df_res['Group'] = df_res.Group.replace({'stroma_poor':'SP','stroma_rich':'SR'})

In [ ]:
#subtype
s_cell = 'celltype2'
d_cell = {'celltype':['tumor', 'immune', 'stromal'],
         'celltype2':['epithelial', 'stromal']}
b_scatterplot = False
for s_group in ['Group','leiden_subtype_group',]: #'leiden_subtype',
    #for s_celltype in sorted(set(df_res.loc[:,s_cell])):
    for s_celltype in d_cell[s_cell]:
        if s_celltype == 'endothelial':
            continue
        df_cell = df_res[df_res.loc[:,s_cell] == s_celltype]
        df_mean = df_cell.loc[:,ls_marker + [s_group,'slide_scene']].groupby([s_group,'slide_scene']).mean()
        if s_celltype == 'epithelial':
            se_myc = df_mean.loc[:,'pMYC']
        df_mean['pMYC_tumor'] = se_myc
        if b_scatterplot:
            for s_marker in ls_marker:
                fig, ax = plt.subplots(figsize=(3,3),dpi=200)
                sns.scatterplot(x='pMYC_tumor', y=s_marker, data=df_mean.reset_index(),ax=ax,hue= s_group,legend=False) #
                r, pvalue = stats.pearsonr(x=df_mean.loc[:,'pMYC_tumor'], y=df_mean.loc[:,s_marker])
                ax.set_title(f'{s_marker} vs pMYC')
                ax.set_ylabel(f'{s_marker} Intensity in {s_celltype}')
                ax.set_xlabel(f'pMYC Intensity in tumor')
                if pvalue > 0.05:
                    plt.close(fig)
                else: 
                    print(f'{s_marker} in {s_celltype} of {s_cell} by {s_group}')
                    ax.set_title(f'{s_marker} vs pMYC\n r = {r:.2f} \np = {pvalue:.5f}')
                    plt.tight_layout()
                    fig.savefig(f'{s_date}/scatterplot_{s_marker}_in_{s_celltype}_of_{s_cell}_by_{s_group}.png')
                    plt.close(fig)
        else:
            for s_marker in ls_marker:
                #s_marker = 'aSMA'#'Ki67'#'pMYC' #'CD45' #
                df_marker = df_mean.loc[:,s_marker].reset_index()
                #test
                lls_result = []
                for s_test in df_marker.loc[:,s_group].unique():
                    ls_result = df_marker.loc[df_marker.loc[:,s_group] == s_test,s_marker].values
                    lls_result.append(ls_result)
                if len(lls_result)==2:
                    try:
                        statistic, pvalue = stats.mannwhitneyu(lls_result[0],lls_result[1])
                    except:
                        pvalue = 1
                        continue
                elif len(lls_result)==3:
                    try:
                        statistic, pvalue = stats.kruskal(lls_result[0],lls_result[1],lls_result[2])
                    except:
                        pvalue = 1
                        continue
                else:
                    pvalue = 1
                #plot    
                ls_order = ['SR_IR','SP','SR_IP']
                if s_group == 'Group':
                    ls_order = ['SR','SP']
                fig,ax=plt.subplots(figsize=(2.5,2.2),dpi=300)
                sns.boxplot(data=df_marker,x=s_group,y=s_marker,ax=ax,fliersize=0,order=ls_order)
                sns.stripplot(data=df_marker,x=s_group,y=s_marker,ax=ax,palette='dark',size=3,order=ls_order)
                ax.set_ylabel(f'{s_marker} in {s_celltype}')
                ax.set_title(f'{s_marker}\n (p = {pvalue:.3f})',fontsize=12,horizontalalignment='center')
                #statistic, pvalue = stats.mannwhitneyu(lls_result[0],lls_result[1])
                plt.tight_layout()
                ##fig.savefig(f'{s_date}/boxplot_{s_cell}_{s_celltype}_{s_group}_{s_marker}.png')
                #break
                if pvalue > 0.05:
                    plt.close(fig)
                else:
                    fig.savefig(f'{s_date}/boxplot_{s_cell}_{s_celltype}_{s_group}_{s_marker}.png')
                    print(f'{s_marker} in {s_celltype} by {s_group}') 
                    plt.close(fig)
                #break
            #break
        
    break

In [ ]:
fig.savefig(f'{s_date}/boxplot_{s_cell}_{s_celltype}_{s_group}_{s_marker}.png')

In [ ]:
#subtype
s_cell = 'celltype2'
d_cell = {'celltype':['tumor', 'immune', 'stromal'],
         'celltype2':['epithelial', 'stromal']}
b_scatterplot = False
for s_group in ['leiden_subtype_group','Group',]: #'leiden_subtype',
    #for s_celltype in sorted(set(df_res.loc[:,s_cell])):
    for s_celltype in d_cell[s_cell]:
        if s_celltype == 'endothelial':
            continue
        df_cell = df_res[df_res.loc[:,s_cell] == s_celltype]
        df_mean = df_cell.loc[:,ls_marker + [s_group,'slide_scene']].groupby([s_group,'slide_scene']).mean()
        if s_celltype == 'epithelial':
            se_myc = df_mean.loc[:,'pMYC']
        df_mean['pMYC_tumor'] = se_myc
        df_mean = df_mean.drop('SR_IP')
        if b_scatterplot:
            for s_marker in ls_marker:
                fig, ax = plt.subplots(figsize=(3,3),dpi=200)
                sns.scatterplot(x='pMYC_tumor', y=s_marker, data=df_mean.reset_index(),ax=ax,hue= s_group,legend=False) #
                r, pvalue = stats.pearsonr(x=df_mean.loc[:,'pMYC_tumor'], y=df_mean.loc[:,s_marker])
                ax.set_title(f'{s_marker} vs pMYC')
                ax.set_ylabel(f'{s_marker} Intensity in {s_celltype}')
                ax.set_xlabel(f'pMYC Intensity in tumor')
                if pvalue > 0.05:
                    plt.close(fig)
                else: 
                    print(f'{s_marker} in {s_celltype} of {s_cell} by {s_group}')
                    ax.set_title(f'{s_marker} vs pMYC\n r = {r:.2f} \np = {pvalue:.5f}')
                    plt.tight_layout()
                    fig.savefig(f'{s_date}/scatterplot_{s_marker}_in_{s_celltype}_of_{s_cell}_by_{s_group}.png')
                    plt.close(fig)
        else:
            for s_marker in ls_marker:
                s_marker = 'EpCAM'#'S100A6'#'Vim' #'aSMA'#'Ki67'#'pMYC' #'CD45' #
                df_marker = df_mean.loc[:,s_marker].reset_index()
                #test
                lls_result = []
                for s_test in df_marker.loc[:,s_group].unique():
                    ls_result = df_marker.loc[df_marker.loc[:,s_group] == s_test,s_marker].values
                    lls_result.append(ls_result)
                if len(lls_result)==2:
                    try:
                        statistic, pvalue = stats.mannwhitneyu(lls_result[0],lls_result[1])
                    except:
                        pvalue = 1
                        continue
                elif len(lls_result)==3:
                    try:
                        statistic, pvalue = stats.kruskal(lls_result[0],lls_result[1],lls_result[2])
                    except:
                        pvalue = 1
                        continue
                else:
                    pvalue = 1
                #plot    
                fig,ax=plt.subplots(figsize=(2.5,2.2),dpi=300)
                sns.boxplot(data=df_marker,x=s_group,y=s_marker,ax=ax,fliersize=0,order=['SR_IR','SP'])
                sns.stripplot(data=df_marker,x=s_group,y=s_marker,ax=ax,palette='dark',size=3,order=['SR_IR','SP'])
                ax.set_ylabel(f'{s_marker} in {s_celltype}')
                ax.set_title(f'{s_marker}\n (p = {pvalue:.3f})',fontsize=12,horizontalalignment='center')
                #statistic, pvalue = stats.mannwhitneyu(lls_result[0],lls_result[1])
                plt.tight_layout()
                ##fig.savefig(f'{s_date}/boxplot_{s_cell}_{s_celltype}_{s_group}_{s_marker}.png')
                break
                if pvalue > 0.05:
                    plt.close(fig)
                else:
                    fig.savefig(f'{s_date}/boxplot_{s_cell}_{s_celltype}_{s_group}_{s_marker}_two.png')
                    print(f'{s_marker} in {s_celltype} by {s_group}') 
                    plt.close(fig)
                #break
            break
        
    break

In [ ]:

#df_marker.sort_values('Vim') #there are several Vim+ Sox9 pos SR IR tissues #D08 is cool looking
#df_marker.sort_values('aSMA')
#df_marker.sort_values(s_marker)

In [ ]:
#df_marker.sort_values(by='pMYC',ascending=False).to_csv('MYC_expression_for_Zinab.csv')

## load  results MIBI (human) <a name="resultsmibi"></a>

celltype, subtype and group

- subtype versus expression




[contents](#contents)

In [ ]:
d_color = {'SR_IR': (0.12156862745098039, 0.4666666666666667, 0.7058823529411765), #blue
 'SP': (1.0, 0.4980392156862745, 0.054901960784313725), #orange
 'SR_IP': (0.17254901960784313, 0.6274509803921569, 0.17254901960784313)} #green
#####d_replace = {'0':'SR_IR','1':'SR_IP','2':'SP','3':'SP'}

In [ ]:
df_mibi = pd.read_csv(f'{rootdir}/results_celltype3_leiden0.1_MIBI.csv',index_col=0)

In [ ]:
df_res = pd.read_csv('results_20211109_MIBI_Celltype_Subtype_Group.csv',index_col=0)
df_res.dropna(inplace=True) #drop no survival info
df_res['leiden_subtype'] = df_res.leiden_subtype.astype('int')
df_res['DAPI_X'] = df_res.DAPI_X.astype('int')
df_res['DAPI_Y'] = df_res.DAPI_Y.astype('int')
ls_marker = df_res.columns[df_res.dtypes=='float64'].tolist()
df_res['leiden_subtype_group'] =  df_res.leiden_subtype.replace({0:'stroma_rich',2:'stroma_rich',1:'stroma_poor'})
df_res['leiden_subtype'] = df_res.leiden_subtype.replace({0: 'SR_IR', 2: 'SR_IP', 1: 'SP'})#{0:'imm_rich',2:'str_rich',1:'epi_rich'})

In [ ]:
df_res.head()

In [ ]:
df_res['slide'] = [item.split('_')[0] for item in df_res.index]

In [ ]:
#df_res['leiden_subtype'] = df_res.slide.map(dict(zip(df_mibi.index.astype('str'),df_mibi.leiden))).replace(d_replace_mibi)

Mouse 0: imm-str-rich
1: epithelial-med
2: epithelial-rich
MIBI 0: imm-rich
1: epithelial rich
2: stromal rich


In [ ]:
df_res['celltype2'] = df_res.gated_celltype.replace({'immune':'stromal','endothelial':'stromal'})

In [ ]:
#subtype
'''s_cell = 'celltype2'#'gated_celltype'
for s_celltype in sorted(set(df_res.loc[:,s_cell])):
    #s_celltype = 'epithelial'
    if s_celltype == 'endothelial':
        continue
    df_cell = df_res[df_res.loc[:,s_cell] == s_celltype]
    for s_group in ['leiden_subtype']: #'leiden_subtype_group',
        df_mean = df_cell.loc[:,ls_marker + [s_group,'slide']].groupby([s_group,'slide']).mean()
        for s_marker in ls_marker:
            #s_marker = 'SMA'
            df_marker = df_mean.loc[:,s_marker].reset_index()
            #test
            lls_result = []
            for s_test in df_marker.loc[:,s_group].unique():
                ls_result = df_marker.loc[df_marker.loc[:,s_group] == s_test,s_marker].values
                lls_result.append(ls_result)
            if len(lls_result)==2:
                try:
                    statistic, pvalue = stats.mannwhitneyu(lls_result[0],lls_result[1])
                except:
                    pvalue = 1
                    continue
            elif len(lls_result)==3:
                try:
                    statistic, pvalue = stats.kruskal(lls_result[0],lls_result[1],lls_result[2])
                except:
                    pvalue = 1
                    continue
            else:
                pvalue = 1
            #plot    
            fig,ax=plt.subplots(figsize=(2.5,2.2),dpi=200)
            sns.boxplot(data=df_marker,x=s_group,y=s_marker,ax=ax,fliersize=0)
            sns.stripplot(data=df_marker,x=s_group,y=s_marker,ax=ax,palette='dark',size=3)
            ax.set_ylabel(f'{s_marker} in {s_celltype}')
            ax.set_title(f'{s_marker}\n (p = {pvalue:.3f})',fontsize=12,horizontalalignment='center')
            statistic, pvalue = stats.mannwhitneyu(lls_result[0],lls_result[1])
            plt.tight_layout()
            #break
            if pvalue > 0.05:
                plt.close(fig)
            else:
                fig.savefig(f'{s_date}/boxplot_MIBI_{s_cell}_{s_celltype}_{s_group}_{s_marker}.png')
                print(f'{s_marker} in {s_celltype} by {s_group}') 
                plt.close(fig)
            #break
        #break
    #break'''

## mibi boxplots

In [ ]:
hrootdir = '/mnt/c/Users/engje/Documents/Data/20211112_AACR'
s_sample = '20211109_MIBI'
resolution = 0.6
df_mib = pd.read_csv(f'{hrootdir}/data/results_{s_sample}_leiden{resolution}_gated_celltype.csv',index_col=0)
df_mib['slide_scene'] = [item.split('_cell')[0] for item in df_mib.index]
df_mib['slide'] = [item.split('_')[0] for item in df_mib.index]
df_mib['slide'] = df_mib.slide.replace({'t1':'1'})
df_mib['celltype3'] = df_mib.gated_celltype.replace({'endothelial':'stromal'})
#df_mib['celltype3'] = df_mib.celltype3.replace({'immune':'stromal'})
#better
df_mib.groupby('celltype3').CD31.count()/len(df_mib)

In [ ]:
s_dataset = 'MIBI'
resolution = 0.1
s_type = 'celltype3'
s_out = f'/mnt/c/Users/engje/Documents/Data/20211112_AACR/data/results_{s_type}_leiden{resolution}_{s_dataset}.csv'
df_p = pd.read_csv(s_out)
df_p['leiden'] = df_p.leiden.astype('str')
d_name = {'0':'stroma_rich','2':'stroma_rich','1':'stroma_poor'}
df_p['subtype'] = df_p.leiden.replace(d_name)
d_named = {'1':'epithelial_rich','2':'stroma_rich','0':'immune_rich'}
df_p['subtype3'] = df_p.leiden.replace(d_named)
df_p.slide =df_p.slide.astype('str')
df_p['leiden_subtype'] = df_p.leiden.replace({'0': 'SR_IR', '2': 'SR_IP', '1': 'SP'})
# add leiden subtype
df_mib['subtype'] = df_mib.slide.map(dict(zip(df_p.slide,df_p.subtype)))
df_mib['subtype3'] = df_mib.slide.map(dict(zip(df_p.slide,df_p.subtype3)))
df_mib['leiden_subtype'] = df_mib.slide.map(dict(zip(df_p.slide,df_p.leiden_subtype)))
df_mib['celltype2'] = df_mib.celltype3.replace({'immune':'stromal'})
df_res = df_mib


In [ ]:
%matplotlib inline

In [ ]:
#expression
ls_marker = ['Beta_catenin', 'CD11b', 'CD11c', 'CD138', 'CD16', 'CD20', 'CD209',
       'CD3', 'CD31', 'CD4', 'CD45', 'CD45RO', 'CD56', 'CD63', 'CD68', 'CD8',
       'dsDNA', 'EGFR', 'FoxP3', 'HLA_DR', 'IDO', 'Keratin17', 'Keratin6',
       'Ki67', 'Lag3', 'MPO', 'p53', 'Pan_Keratin', 'PD_L1', 'PD1', 'SMA',
       'Vimentin']
len(ls_marker)
s_cell = 'celltype2'
for s_group in ['leiden_subtype']:
    print(s_group)
    for s_celltype in sorted(set(df_res.loc[:,s_cell])):
        print(s_celltype)
        if s_celltype == 'endothelial':
            continue
        df_cell = df_res[df_res.loc[:,s_cell] == s_celltype]
        df_mean = df_cell.loc[:,ls_marker + [s_group,'slide']].groupby([s_group,'slide']).mean()

        for s_marker in ls_marker:
            s_marker = 'SMA'#'Pan_Keratin'#'Keratin6'#'Beta_catenin'#'PD1'#'PD_L1'#'SMA'#'Vimentin'#'Ki67'#
            df_marker = df_mean.loc[:,s_marker].reset_index()
            #test
            lls_result = []
            for s_test in df_marker.loc[:,s_group].unique():
                ls_result = df_marker.loc[df_marker.loc[:,s_group] == s_test,s_marker].values
                lls_result.append(ls_result)
            if len(lls_result)==2:
                try:
                    statistic, pvalue = stats.mannwhitneyu(lls_result[0],lls_result[1])
                except:
                    pvalue = 1
                    continue
            elif len(lls_result)==3:
                try:
                    statistic, pvalue = stats.kruskal(lls_result[0],lls_result[1],lls_result[2])
                except:
                    pvalue = 1
                    continue
            elif len(lls_result)==4:
                try:
                    statistic, pvalue = stats.kruskal(lls_result[0],lls_result[1],lls_result[2],lls_result[3])
                except:
                    pvalue = 1
                    continue
            else:
                pvalue = 1
            #plot    
            fig,ax=plt.subplots(figsize=(2.5,2.2),dpi=300)
            sns.boxplot(data=df_marker,x=s_group,y=s_marker,ax=ax,fliersize=0,order=['SR_IR','SP','SR_IP'])
            sns.stripplot(data=df_marker,x=s_group,y=s_marker,ax=ax,palette='dark',size=3,order=['SR_IR','SP','SR_IP'])
            ax.set_ylabel(f'{s_marker} in {s_celltype}')
            ax.set_title(f'{s_marker}\n (p = {pvalue:.3f})',fontsize=12,horizontalalignment='center')
            labels = [item.get_text() for item in ax.get_xticklabels()]
            if len(lls_result)==3:
                ax.set_xticklabels([item.replace('_rich',f'') for item in labels])
                #ax.set_xlabel('(-rich)')
            plt.tight_layout()
            #fig.savefig(f'{rootdir}/{s_date}/boxplot_MIBI_{s_cell}_{s_celltype}_{s_group}_{s_marker}.png')
            break
            if pvalue > 0.05:
                plt.close(fig)
            else:
                fig.savefig(f'{rootdir}/MIBI/boxplot_MIBI_{s_cell}_{s_celltype}_{s_group}_{s_marker}.png')
                print(f'{s_marker} in {s_celltype} by {s_group}') 
                #plt.close(fig)
                break
        #break
    break
    

In [ ]:
fig.savefig(f'{rootdir}/MIBI/boxplot_MIBI_{s_cell}_{s_celltype}_{s_group}_{s_marker}.png')

In [ ]:
#expression
ls_marker = ['Beta_catenin', 'CD11b', 'CD11c', 'CD138', 'CD16', 'CD20', 'CD209',
       'CD3', 'CD31', 'CD4', 'CD45', 'CD45RO', 'CD56', 'CD63', 'CD68', 'CD8',
       'dsDNA', 'EGFR', 'FoxP3', 'HLA_DR', 'IDO', 'Keratin17', 'Keratin6',
       'Ki67', 'Lag3', 'MPO', 'p53', 'Pan_Keratin', 'PD_L1', 'PD1', 'SMA',
       'Vimentin']
len(ls_marker)
s_cell = 'celltype2'
for s_group in ['leiden_subtype']:
    print(s_group)
    for s_celltype in sorted(set(df_res.loc[:,s_cell])):
        print(s_celltype)
        if s_celltype == 'endothelial':
            continue
        df_cell = df_res[df_res.loc[:,s_cell] == s_celltype]
        df_mean = df_cell.loc[:,ls_marker + [s_group,'slide']].groupby([s_group,'slide']).mean()
        df_mean = df_mean.drop('SR_IP')
        for s_marker in ls_marker:
            s_marker = 'Vimentin'#'Beta_catenin'#'Keratin6'#'Pan_Keratin'#'PD1'#'PD_L1'#'SMA'#'Ki67'#
            df_marker = df_mean.loc[:,s_marker].reset_index()
            #test
            lls_result = []
            for s_test in df_marker.loc[:,s_group].unique():
                ls_result = df_marker.loc[df_marker.loc[:,s_group] == s_test,s_marker].values
                lls_result.append(ls_result)
            if len(lls_result)==2:
                try:
                    statistic, pvalue = stats.mannwhitneyu(lls_result[0],lls_result[1])
                except:
                    pvalue = 1
                    continue
            elif len(lls_result)==3:
                try:
                    statistic, pvalue = stats.kruskal(lls_result[0],lls_result[1],lls_result[2])
                except:
                    pvalue = 1
                    continue
            elif len(lls_result)==4:
                try:
                    statistic, pvalue = stats.kruskal(lls_result[0],lls_result[1],lls_result[2],lls_result[3])
                except:
                    pvalue = 1
                    continue
            else:
                pvalue = 1
            #plot    
            fig,ax=plt.subplots(figsize=(2.5,2.2),dpi=300)
            sns.boxplot(data=df_marker,x=s_group,y=s_marker,ax=ax,fliersize=0,order=['SR_IR','SP'])
            sns.stripplot(data=df_marker,x=s_group,y=s_marker,ax=ax,palette='dark',size=3,order=['SR_IR','SP'])
            ax.set_ylabel(f'{s_marker} in {s_celltype}')
            ax.set_title(f'{s_marker}\n (p = {pvalue:.3f})',fontsize=12,horizontalalignment='center')
            labels = [item.get_text() for item in ax.get_xticklabels()]
            if len(lls_result)==3:
                ax.set_xticklabels([item.replace('_rich',f'') for item in labels])
                #ax.set_xlabel('(-rich)')
            plt.tight_layout()
            #fig.savefig(f'{rootdir}/{s_date}/boxplot_MIBI_{s_cell}_{s_celltype}_{s_group}_{s_marker}.png')
            #break
            if pvalue > 0.05:
                plt.close(fig)
            else:
                fig.savefig(f'{rootdir}/MIBI/boxplot_MIBI_{s_cell}_{s_celltype}_{s_group}_{s_marker}_two.png')
                print(f'{s_marker} in {s_celltype} by {s_group}') 
                #plt.close(fig)
                break
        break
    break
    

In [ ]:
#df_marker.sort_values(by='Beta_catenin') #7 vs 17
df_marker.sort_values(by='Vimentin') #7 vs 29. there are examples of vim coexpressing with beta catenin and also just lateral bleed through (eg 40)
#point 29, high levels of vim and aSMA

In [ ]:
fig.savefig(f'{rootdir}/MIBI/boxplot_MIBI_{s_cell}_{s_celltype}_{s_group}_{s_marker}.png')

# mibi cell types <a name="mibiann"></a>

clustering results




[contents](#contents)

In [ ]:
s_sample = '20211109_MIBI'

In [ ]:
#colors
se_annot = df_res.loc[:,['leiden','gated_celltype']].groupby('leiden').agg(pd.Series.mode)
d_annot = dict(zip(df_res.leiden.astype('str'),df_res.leiden_celltype))
d_color_ct = {'immune':'C0', 'epithelial':'C1','stromal':'C2'}
row_colors = df_plot.index.map(se_annot.to_dict()['gated_celltype']).map(d_color_ct)
ls_drop_index= ['DAPI_X','DAPI_Y']
df_plot = df_res.groupby('leiden').mean().drop(ls_drop_index,axis=1).dropna()
df_plot.index.name = f'leiden'
g = sns.clustermap(df_plot,z_score=1,figsize=(8,9),cmap='viridis',method='ward',cbar_pos=(.01, .91, .05, .15),
                   vmin=-2,vmax=2,dendrogram_ratio=0.1,row_colors=row_colors)
g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize = 14)
g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_ymajorticklabels(), fontsize = 14,rotation=0)
for label in d_color_ct.keys():
    g.ax_row_dendrogram.bar(0, 0, color=d_color_ct[label],
                            label=label, linewidth=0)
g.ax_row_dendrogram.legend(loc="right", ncol=1,fontsize=12)
g.savefig(f'{rootdir}/MIBI/{s_sample}_clustermap_leiden_celltypes.png',dpi=300)
marker_genes = df_plot.iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_plot.iloc[g.dendrogram_row.reordered_ind,:].index.tolist()

In [ ]:
df_res.head()
from mplex_image import analyze
df_pos = analyze.celltype_to_bool(df_res,'gated_celltype')
df_pos.columns = [str(item) for item in df_pos.columns]

sns.palplot(mpl.cm.Paired.colors)

In [ ]:
s_slide = '7'
ls_com = df_res[df_res.slide_scene==s_slide].groupby('leiden').count().CD31.sort_values(ascending=False)[0:9].index.to_list()
df_res['slide_scene'] = df_res.slide.astype('str')
ls_colors = [mpl.cm.Paired.colors[0],mpl.cm.Paired.colors[8],
           mpl.cm.Paired.colors[10],mpl.cm.Paired.colors[9],
           mpl.cm.Paired.colors[3],mpl.cm.Paired.colors[1],
           mpl.cm.Paired.colors[5],mpl.cm.Paired.colors[7],
           mpl.cm.Paired.colors[4]]
d_color = dict(zip(ls_com,ls_colors))
d_color = {'endothelial':mpl.cm.Paired.colors[1],
           'epithelial':'orange',
           'immune':'magenta',
           'stromal':'green'}
ls_com = d_color.keys()

#plot all groups spatially - leiden 
ls_out = []

#for s_slide in sorted(set(df_p.slide_scene)):
for s_slides in ['RS-mTMA-5_sceneB08','RS-mTMA-5_sceneI03','RS-mTMA-5_sceneH11','RS-mTMA-5_sceneA05']:
    fig,ax = plt.subplots(figsize=(6,3.5),dpi=300)
    #plot negative cells
    df_scene = df_res[df_res.slide_scene==s_slide]
    ax.scatter(data=df_scene,x='DAPI_X',y='DAPI_Y',color='silver',s=.5,label=f'')
    #for idxs, s_color_int in enumerate(range(len(df_pos.columns))):
    for idxs, s_color in enumerate(ls_com): 
        s_color = str(s_color)
        if len(df_res[(df_res.slide_scene==s_slide) & (df_pos.loc[:,s_color])])>=10:
            #plot positive cells
            ax.scatter(data=df_res[(df_res.slide_scene==s_slide) & (df_pos.loc[:,s_color])],x='DAPI_X',y='DAPI_Y',
                       label=s_color,
                       s=.5,color=d_color[s_color])
        else:
            ls_out.append(s_color)
    ax.set_title(f"{s_slide}", fontsize=16)
    ax.axis('equal')
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)
    plt.tight_layout()
    plt.legend(markerscale=10,framealpha=.5,bbox_to_anchor=(.75,.8),fontsize=8) 
    fig.savefig(f'{rootdir}/SpatialPlots/MIBI_{s_slide}_leiden_scatterplot.png')
    break

## mibi subtyping

match to AACR

In [ ]:
#just use clustering cell types
#my MIBI celltypes
s_sample = '20211109_MIBI'
resolution = 0.6
df_mib = pd.read_csv(f'results_{s_sample}_leiden{resolution}_gated_celltype.csv',index_col=0)
df_mib['slide_scene'] = [item.split('_cell')[0] for item in df_mib.index]
df_mib['slide'] = [item.split('_')[0] for item in df_mib.index]
df_mib['slide'] = df_mib.slide.replace({'t1':'1'})
df_mib['celltype3'] = df_mib.gated_celltype.replace({'endothelial':'stromal'})
#df_mib['celltype3'] = df_mib.celltype3.replace({'immune':'stromal'})
#better
df_mib.groupby('celltype3').CD31.count()/len(df_mib)

In [ ]:
#MIBI
s_dataset = 'MIBI'
s_type = 'celltype3'
df = df_mib.loc[:,['CD31',s_type,'slide']].groupby([s_type,'slide']).CD31.count()/df_mib.loc[:,['CD31','slide']].groupby(['slide']).CD31.count()
df_prop = df.unstack().T.fillna(0)
ls_col = df_prop.columns.tolist()
adata = sc.AnnData(df_prop)
adata.raw = adata
#reduce dimensionality
sc.tl.pca(adata, svd_solver='auto')
#umap
for n_neighbors in [5]: #5
    results_file = f'{rootdir}/{s_date}/{s_sample}_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}_{s_dataset}.h5ad'
    if not os.path.exists(results_file):
        print('calc umap')
        # calculate neighbors 
        sc.pp.neighbors(adata, n_neighbors=n_neighbors) #, method='rapids'
        sc.tl.umap(adata)
    else:
        print('loading umap')
        adata = sc.read_h5ad(results_file)
    #color by markers
    
    figname = f"Umap_markers_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}_MIBI.png"
    title=figname.split('.png')[0].replace('_',' ')
    sc.pl.umap(adata, color=ls_col,vmin='p1.5',vmax='p99.5',ncols=4,save=figname,size=250)
    #survival
    #save results
    if not os.path.exists(results_file):
        adata.write(results_file)
    
    break

In [ ]:
for resolution in [0.1]: #0.2
    results_file = f'{rootdir}/{s_date}/{s_sample}_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}_{s_dataset}.h5ad'
    adata = sc.read_h5ad(results_file)
    sc.tl.leiden(adata,resolution=resolution)
    fig,ax = plt.subplots(figsize=(2,1.5),dpi=200)
    figname=f'leiden_{n_neighbors}_{len(ls_col)}_{resolution}_{s_dataset}.png'
    sc.pl.umap(adata, color='leiden',ax=ax,title=figname.split('.png')[0].replace('_',' '),wspace=.25,save=figname,size=40)
    break

In [ ]:
#clustermap
df_p = pd.DataFrame(data=adata.raw.X,index=adata.obs.index,columns=adata.var.index)
df_p['leiden'] = adata.obs['leiden']
d_replace = {}
df_plot = df_p.groupby('leiden').mean()
df_plot.index.name = f'leiden {resolution}'
g = sns.clustermap(df_plot,figsize=(4,3.5),cmap='viridis',z_score=1,vmin=-1.5,vmax=1.5
                   )
#g.fig.suptitle(f'leiden {resolution}',x=.9) 
g.savefig(f'{rootdir}/{s_date}/{s_sample}_clustermap_{s_type}_subtype_{resolution}_{s_dataset}.png',dpi=200)
marker_genes = df_p.groupby('leiden').mean().iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_p.groupby('leiden').mean().iloc[g.dendrogram_row.reordered_ind,:].index.tolist()
d_lei = dict(zip(adata.obs.index,adata.obs.leiden))
df_prop['leiden'] = df_prop.index.map(d_lei)
print(len(df_p))

In [ ]:
d_replace_mib = {'0':'SR_IR','2':'SR_IP','1':'SP'}

In [ ]:
#clustermap
rdir = '/mnt/c/Users/engje/Documents/Data/cmIF_2021-07-07_RS-mTMA'
df_plot = df_p[df_p.leiden!='2'].groupby('leiden').mean().dropna()
df_plot = df_p.groupby('leiden').mean().rename(d_replace_mib)
df_plot.index.name = f'leiden {resolution}'
fig,ax=plt.subplots(figsize=(3.2,2),dpi=300)
sns.heatmap(df_plot.loc[['SR_IR','SR_IP','SP'],:],cmap='viridis',ax=ax #,z_score=1,vmin=-1.5,vmax=1.5,figsize=(4,3.5),
                   )
plt.tight_layout()
fig.savefig(f'{rdir}/MIBI/{s_sample}_heatmap_{s_type}_subtype_{resolution}_{s_dataset}.png',dpi=300)

In [ ]:
#
d_color = {'SR_IR': (0.12156862745098039, 0.4666666666666667, 0.7058823529411765), #blue
 'SP': (1.0, 0.4980392156862745, 0.054901960784313725), #orange
 'SR_IP': (0.17254901960784313, 0.6274509803921569, 0.17254901960784313)} #green
row_colors = df_p.index.map(dict(zip(adata.obs.index,adata.obs.leiden.replace(d_replace_mib)))).map(d_color)

g = sns.clustermap(df_p.drop('leiden',axis=1),figsize=(3.6,4),cmap='viridis',row_colors=row_colors,#z_score=1
        )
#g.fig.suptitle(f'leiden {resolution}',x=.9) 
for label in adata.obs.leiden.replace(d_replace_mib).unique():
    g.ax_row_dendrogram.bar(0, 0, color=d_color[label],
                            label=label, linewidth=0)
g.ax_row_dendrogram.legend(loc="right", ncol=1)
g.savefig(f'{rdir}/MIBI/{s_sample}_{s_type}_clustermap_Group_leiden{resolution}_leiden.png',dpi=300)
marker_genes = df_prop.iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_prop.iloc[g.dendrogram_row.reordered_ind,:].index.tolist()

In [ ]:
#load annotation
import lifelines
from lifelines import KaplanMeierFitter
from lifelines.statistics import multivariate_logrank_test
from lifelines import CoxPHFitter
from scipy import stats
b_recur = True
df_surv_mib = pd.read_csv(f'/mnt/c/Users/engje/Documents/Data/20211112_AACR/data/MIBI_clinical_data.csv',index_col=0)
df_surv_mib.index =df_surv_mib.index.astype('str')
df = df_prop.merge(df_surv_mib.loc[:,['Survival','Survival_time']],left_index=True,right_index=True)
if b_recur:
    df = df_prop.merge(df_surv_mib.loc[:,['Recurrence','Recurrence_time']],left_index=True,right_index=True)
    s_recur = 'recurrence'
else:
    s_recur = 'survival'

In [ ]:
ls_group =  sorted(df.leiden.unique()) #['1',  '2'] # ['0',  '2'] #
if b_recur:
    T = df.loc[df.leiden.isin(ls_group),'Recurrence_time']     ## time to event
    E = df.loc[df.leiden.isin(ls_group),'Recurrence'] 
else:
    T = df.loc[df.leiden.isin(ls_group),'Survival_time']     ## time to event
    E = df.loc[df.leiden.isin(ls_group),'Survival']      ## event occurred or censored
groups = df.loc[df.leiden.isin(ls_group),'leiden']  
kmf1 = KaplanMeierFitter() ## instantiate the class to create an object
fig, ax = plt.subplots(figsize=(3,3),dpi=200)
for s_group in ls_group:
    i1 = (groups == s_group)
    kmf1.fit(T[i1], E[i1], label=s_group)    ## fit thedata
    kmf1.plot(ax=ax,ci_show=False)
    #print(kmf1.median_survival_time_)
results = multivariate_logrank_test(event_durations=T, groups=groups, event_observed=E)
ax.set_title(f'{s_type} {resolution} \n p={results.summary.p[0]:.2}')
ax.set_ylim(-.05,1.05)
ax.legend(loc='upper right')
plt.tight_layout()
fig.savefig(f'{rdir}/MIBI/KM_{resolution}_{s_type}_{s_dataset}_{s_recur}.png',dpi=200)

In [ ]:
d_named = {'1':'epithelial_rich','2':'stroma_rich','0':'immune_rich'}
d_named = {'1':'str_poor','2':'str_mid','0':'imm-str_rich'}
d_named = {'0': 'SR_IR', '2': 'SR_IP', '1': 'SP'}
df['subtype'] = df.leiden.replace(d_named)
ls_group = sorted(df.subtype.unique()) # ['0',  '2'] #

groups = df.loc[df.subtype.isin(ls_group),'subtype']  
kmf1 = KaplanMeierFitter() ## instantiate the class to create an object
fig, ax = plt.subplots(figsize=(3.5,3),dpi=400)
for s_group in ['SR_IR','SP','SR_IP']:#ls_group:
    i1 = (groups == s_group)
    kmf1.fit(T[i1], E[i1], label=s_group)    ## fit thedata
    kmf1.plot(ax=ax,ci_show=False,show_censors=True,censor_styles={'linewidth':.5})
    #print(kmf1.median_survival_time_)
results = multivariate_logrank_test(event_durations=T, groups=groups, event_observed=E)
ax.set_title(f'Overall Survival \n p={results.summary.p[0]:.2}',fontsize=14)
if b_recur:
    ax.set_title(f'Recurrence-Free Survival \n p={results.summary.p[0]:.2}',fontsize=14)
ax.set_ylim(-.05,1.05)
ax.legend(loc='lower left')
ax.set_xlabel('Days')
ax.set_ylabel('Fraction Alive')
if b_recur:
    ax.set_ylabel('Fraction Recurrence-Free')
plt.tight_layout()
#fig.savefig(f'{rootdir}/{s_date}/KM_{resolution}_{s_type}_{s_dataset}_{s_recur}_named.png',dpi=300)
fig.savefig(f'{rdir}/MIBI/KM_{resolution}_{s_type}_{s_dataset}_{s_recur}_named.png',dpi=300)
print(len(df))

In [ ]:
#collapes
#d_col = {'2':'0'}
d_name = {'0':'SR','2':'SR','1':'SP'}
if b_recur:
    d_name  = {'0':'stroma_rich','2':'stroma_rich','1':'stroma_poor'} #d_name = {'0':'immune_rich','2':'immune_poor','1':'immune_poor'}
    #d_name  = {'0':'stroma_poor','2':'stroma_rich','1':'stroma_poor'} #imm rich = str poor
df['subtype'] = df.leiden.replace(d_name)
ls_group = sorted(df.subtype.unique()) # ['0',  '2'] #

groups = df.loc[df.subtype.isin(ls_group),'subtype']  
kmf1 = KaplanMeierFitter() ## instantiate the class to create an object
fig, ax = plt.subplots(figsize=(3,3),dpi=200)
for s_group in ls_group:
    i1 = (groups == s_group)
    kmf1.fit(T[i1], E[i1], label=s_group)    ## fit thedata
    kmf1.plot(ax=ax,ci_show=False,show_censors=True)
    #print(kmf1.median_survival_time_)
results = multivariate_logrank_test(event_durations=T, groups=groups, event_observed=E)
ax.set_title(f'{s_type} {resolution} \n p={results.summary.p[0]:.2}')
ax.set_ylim(-.05,1.05)
ax.legend(loc='upper right')
plt.tight_layout()
fig.savefig(f'{rdir}/MIBI/KM_{resolution}_{s_type}_{s_dataset}_{s_recur}_colapsed.png',dpi=200)
print(len(df))

## OLD redo subtypeing

locallly (exacloud down)

In [ ]:
s_sample = '20211109_MIBI'
resolution = 0.6
df_p = pd.read_csv(f'results_{s_sample}_leiden{resolution}_gated_celltype.csv',index_col=0)
df_p['slide_scene'] = [item.split('_cell')[0] for item in df_p.index]
df_p['slide'] = [item.split('_')[0] for item in df_p.index]
s_type = 'gated_celltype' #'leiden' #
df = df_p.loc[:,['CD31',s_type,'slide']].groupby([s_type,'slide']).CD31.count()/df_p.loc[:,['CD31','slide']].groupby(['slide']).CD31.count()
df_prop = df.unstack().T.fillna(0)
ls_col = df_prop.columns.tolist()
adata = sc.AnnData(df_prop)
adata.raw = adata
#reduce dimensionality
sc.tl.pca(adata, svd_solver='auto')
fig,ax=plt.subplots(figsize=(3.5,5))
sc.pl.highest_expr_genes(adata, n_top=48,ax=ax,save=f'Expression_{len(ls_col)}_{s_type}.png')
plt.tight_layout()
sc.pl.pca_variance_ratio(adata,n_pcs=32, log=True)

#umap
for n_neighbors in [5]: #,30
    results_file = f'{s_date}/{s_sample}_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.h5ad'
    print('calc umap')
    # calculate neighbors 
    sc.pp.neighbors(adata, n_neighbors=n_neighbors) #, method='rapids'
    sc.tl.umap(adata)

    #color by markers
    
    figname = f"Umap_markers_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}_MIBI.png"
    title=figname.split('.png')[0].replace('_',' ')
    sc.pl.umap(adata, color=ls_col,vmin='p1.5',vmax='p99.5',ncols=4,save=figname,size=250)
    #survival
    '''
    adata.obs['slide'] = adata.obs.index.tolist()
    adata.obs['Group'] = adata.obs.slide.map(d_a).fillna('unknown')
    #adata.obs['Group'] = adata.obs.index.map(d_a).fillna('unknown')
            
    figname = f"Umap_Group_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
    title=figname.split('.png')[0].replace('_',' ')
    fig,ax = plt.subplots(figsize=(3,2), dpi=200)
    sc.pl.umap(adata, color='Group',wspace=.25,save=figname,ax=ax,title=title,size=50)
    
    # time
    figname = f"Umap_survival_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
    title=figname.split('.png')[0].replace('_',' ')
    fig,ax = plt.subplots(figsize=(3,2), dpi=200)
    adata.obs['survival'] =adata.obs.slide.map(dict(zip(df_a.index.astype('str'),df_a.Survival_time)))
    sc.pl.umap(adata, color='survival',size=50,ax=ax,title=title,save=figname)

    #save results
    if not os.path.exists(results_file):
        adata.write(results_file)
    '''
    break

In [ ]:
#leiden
for resolution in [0.2]: #0.3,
    results_file = f'{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}_leiden{resolution}.h5ad'
    sc.tl.leiden(adata,resolution=resolution)
    adata.write(results_file)
    fig,ax = plt.subplots(figsize=(2,2),dpi=200)
    figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}.png'
    sc.pl.umap(adata, color='leiden',ax=ax,title=figname.split('.png')[0].replace('_',' '),wspace=.25,save=figname,size=40)
    break

In [ ]:
#clustermap
df_p = pd.DataFrame(data=adata.raw.X,index=adata.obs.index,columns=adata.var.index)
df_p['leiden'] = adata.obs['leiden']
d_replace = {}
df_plot = df_p.groupby('leiden').mean()
df_plot.index.name = f'leiden {resolution}'
g = sns.clustermap(df_plot,z_score=1,figsize=(4,3.5),cmap='viridis',
                   vmin=-1.5,vmax=1.5)
#g.fig.suptitle(f'leiden {resolution}',x=.9) 
g.savefig(f'{rootdir}/{s_date}/{s_sample}_clustermap_{s_type}_subtype_{resolution}_MIBI.png',dpi=200)
marker_genes = df_p.groupby('leiden').mean().iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_p.groupby('leiden').mean().iloc[g.dendrogram_row.reordered_ind,:].index.tolist()

In [ ]:
d_replace = {'0':'SR_IR','1':'SR_IP','2':'SP','3':'SP'}
d_replace = {'3':'2'}
d_replace = {'0':'SR_IR','1':'SR_IP','2':'SP','3':'SP'}
df_plot = df_p.groupby(df_p.leiden.replace(d_replace)).mean()
df_plot.index.name = f'leiden {resolution}'
fig,ax=plt.subplots(figsize=(3.2,2),dpi=300)
sns.heatmap(df_plot,cmap='viridis',ax=ax#z_score=1,vmin=-1.5,vmax=1.5
                   )
plt.tight_layout()
fig.savefig(f'{rootdir}/{s_date}/{s_sample}_heatmap_{s_type}_subtype_{resolution}.png',dpi=300)

In [ ]:
#sns.clustermap(df_prop)
s_type = 'gated_celltype' #'leiden' #
df = df_p.loc[:,['CD31',s_type,'slide']].groupby([s_type,'slide']).CD31.count()/df_p.loc[:,['CD31','slide']].groupby(['slide']).CD31.count()
df_prop = df.unstack().T.fillna(0)
d_lei = dict(zip(adata.obs.index,adata.obs.leiden))
d_color = dict(zip(sorted(adata.obs.leiden.unique()),sns.color_palette()[0:3]))
row_colors = df_prop.index.map(d_lei).map(d_color)

g = sns.clustermap(df_prop,figsize=(5,15),cmap='viridis',row_colors=row_colors
        )
for label in d_color.keys():
    g.ax_row_dendrogram.bar(0, 0, color=d_color[label],
                            label=label, linewidth=0)
g.ax_row_dendrogram.legend(loc="right", ncol=1)
g.savefig(f'{rootdir}/{s_date}/{s_sample}_clustermap_Group_MIBI.png',dpi=200)
marker_genes = df_prop.iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_prop.iloc[g.dendrogram_row.reordered_ind,:].index.tolist()

In [ ]:
#categories_order
ls_epi = ['15', '22', '18', '24', '7', '14', '21', '23', '27', '26', '31',]
ls_str = [ '38', '11', '20', '34', '6', '25', '33', '19', '8', '32', '35', '16', '28', '2', '36', '1', '3', '5', '12', '9', '17', '39', '29', '37', '41', '10', '40', '13', '4']

## OLD mibi survival

In [ ]:
import lifelines
from lifelines import KaplanMeierFitter
from lifelines.statistics import multivariate_logrank_test
from lifelines import CoxPHFitter

In [ ]:
df_a = pd.read_csv(f'{rootdir}/clinical_data.csv',index_col=0)
#load annotation
df_a['Group'] = pd.qcut(x=df_a.loc[:,'Survival_time'],q=2,labels=['short','long'])
d_a = dict(zip(df_a.index.astype('str'),df_a.Group))

df_prop['leiden'] = df_prop.index.map(d_lei)

df_a.index = df_a.index.astype('str')
df = df_prop.merge(df_a.loc[:,['Survival','Survival_time']],left_index=True,right_index=True)

In [ ]:
T = df['Survival_time']     ## time to event
E = df['Survival']      ## event occurred or censored
groups = df.loc[:,'leiden']  
kmf1 = KaplanMeierFitter() ## instantiate the class to create an object
fig, ax = plt.subplots(figsize=(3,3),dpi=200)
for s_group in sorted(groups.unique()):
    i1 = (groups == s_group)
    kmf1.fit(T[i1], E[i1], label=s_group)    ## fit thedata
    kmf1.plot(ax=ax,ci_show=False)
    #print(kmf1.median_survival_time_)
results = multivariate_logrank_test(event_durations=T, groups=groups, event_observed=E)
ax.set_title(f'{s_type} {resolution} \n p={results.summary.p[0]:.2}')
ax.legend(loc='upper right')
plt.tight_layout()
fig.savefig(f'{s_date}/KM_{resolution}_{s_type}.png',dpi=200)

In [ ]:
df.loc[df.index.isin(ls_epi),'Group'] ='stroma_poor'
df.loc[df.index.isin(ls_str),'Group'] ='stroma_rich'
s_type = 'Group'
groups = df.loc[:,'Group']  
kmf1 = KaplanMeierFitter() ## instantiate the class to create an object
fig, ax = plt.subplots(figsize=(3,3),dpi=200)
for s_group in sorted(groups.unique()):
    i1 = (groups == s_group)
    kmf1.fit(T[i1], E[i1], label=s_group)    ## fit thedata
    kmf1.plot(ax=ax,ci_show=False)
    #print(kmf1.median_survival_time_)
results = multivariate_logrank_test(event_durations=T, groups=groups, event_observed=E)
ax.set_title(f'{s_type} {resolution} \n p={results.summary.p[0]:.2}')
ax.legend(loc='upper right')
plt.tight_layout()
fig.savefig(f'{s_date}/KM_{resolution}_{s_type}_MIBI.png',dpi=200)

In [ ]:
df_p['Group'] = df_p.slide.map(dict(zip(df.index,df.Group)))

In [ ]:
df_p['leiden_subtype'] = df_p.slide.map(dict(zip(df.index,df.leiden)))

In [ ]:
s_out_mib = 'results_20211109_MIBI_Celltype_Subtype_Group.csv'
#df_p.to_csv(s_out_mib)

In [ ]:
df_p.head()

In [ ]:
df_res = pd.read_csv('results_20211109_MIBI_leiden0.6_gated_celltype.csv',index_col=0)

In [ ]:
df_res

## stop

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_species_distributions
from sklearn.neighbors import KernelDensity

In [ ]:
try:
    from mpl_toolkits.basemap import Basemap

    basemap = True
except ImportError:
    basemap = False

In [ ]:
def construct_grids(batch):
    """Construct the map grid from the batch object

    Parameters
    ----------
    batch : Batch object
        The object returned by :func:`fetch_species_distributions`

    Returns
    -------
    (xgrid, ygrid) : 1-D arrays
        The grid corresponding to the values in batch.coverages
    """
    # x,y coordinates for corner cells
    xmin = batch.x_left_lower_corner + batch.grid_size
    xmax = xmin + (batch.Nx * batch.grid_size)
    ymin = batch.y_left_lower_corner + batch.grid_size
    ymax = ymin + (batch.Ny * batch.grid_size)

    # x coordinates of the grid cells
    xgrid = np.arange(xmin, xmax, batch.grid_size)
    # y coordinates of the grid cells
    ygrid = np.arange(ymin, ymax, batch.grid_size)

    return (xgrid, ygrid)

In [ ]:
# Get matrices/arrays of species IDs and locations
data = fetch_species_distributions()
species_names = ["Bradypus Variegatus", "Microryzomys Minutus"]

In [ ]:
data

In [ ]:

Xtrain = np.vstack([data["train"]["dd lat"], data["train"]["dd long"]]).T
ytrain = np.array(
    [d.decode("ascii").startswith("micro") for d in data["train"]["species"]],
    dtype="int",
)
Xtrain *= np.pi / 180.0  # Convert lat/long to radians

# Set up the data grid for the contour plot
xgrid, ygrid = construct_grids(data)

In [ ]:
# Set up the data grid for the contour plot
xgrid, ygrid = construct_grids(data)
X, Y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])
land_reference = data.coverages[6][::5, ::5]
land_mask = (land_reference > -9999).ravel()

In [ ]:
xy = np.vstack([Y.ravel(), X.ravel()]).T
xy = xy[land_mask]
xy *= np.pi / 180.0

In [ ]:
%matplotlib inline
# Plot map of South America with distributions of each species
fig = plt.figure()
fig.subplots_adjust(left=0.05, right=0.95, wspace=0.05)

for i in range(2):
    plt.subplot(1, 2, i + 1)

    # construct a kernel density estimate of the distribution
    print(" - computing KDE in spherical coordinates")
    kde = KernelDensity(
        bandwidth=0.04, metric="haversine", kernel="gaussian", algorithm="ball_tree"
    )
    kde.fit(Xtrain[ytrain == i])

    # evaluate only on the land: -9999 indicates ocean
    Z = np.full(land_mask.shape[0], -9999, dtype="int")
    Z[land_mask] = np.exp(kde.score_samples(xy))
    Z = Z.reshape(X.shape)

    # plot contours of the density
    levels = np.linspace(0, Z.max(), 25)
    plt.contourf(X, Y, Z, levels=levels, cmap=plt.cm.Reds)

    if basemap:
        print(" - plot coastlines using basemap")
        m = Basemap(
            projection="cyl",
            llcrnrlat=Y.min(),
            urcrnrlat=Y.max(),
            llcrnrlon=X.min(),
            urcrnrlon=X.max(),
            resolution="c",
        )
        m.drawcoastlines()
        m.drawcountries()
    else:
        print(" - plot coastlines from coverage")
        plt.contour(
            X, Y, land_reference, levels=[-9998], colors="k", linestyles="solid"
        )
        plt.xticks([])
        plt.yticks([])

    plt.title(species_names[i])

plt.show()

In [ ]:
import shapely
polygon = [(1.0, 1.0), (2.0, 1.0), (2.0, 2.0), (1,2), (1, 1)]
shapely_poly = shapely.geometry.Polygon(polygon)
line = [(1, 2), (2,1)]
shapely_line = shapely.geometry.LineString(line)
intersection_line = list(shapely_poly.intersection(shapely_line).coords)

In [ ]:
from shapely.geometry import Point

point = Point(0, -10)
square = point.buffer(0.5).envelope

fig, ax = plt.subplots(figsize=(5,5))
gpd.GeoSeries(square).plot(ax=ax)
gpd.GeoSeries(point).plot(ax=ax, color = "black",markersize=30)
plt.grid()
plt.show()

In [ ]:
import geopandas as gpd